In [1]:
import numpy as np
import os
import random
from PIL import Image
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from keras.layers import Input, Conv2D
from keras.models import Model
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, UpSampling2D, Flatten, Dense, Reshape
import os
from PIL import Image, ImageDraw
import random

In [24]:
import os
from PIL import Image

# پوشه‌ای برای ذخیره‌سازی تصاویر
output_folder = "data_color_2"
os.makedirs(output_folder, exist_ok=True)

# ابعاد تصویر
base_size = (2, 2)  # ابعاد اصلی
final_size = (32, 32)  # ابعاد بزرگ‌شده (برای مشاهده بهتر)

# حداکثر تعداد رنگ‌ها
max_colors = 200000

# تولید رنگ‌ها به ترتیب
all_colors = [(r, g, b) for r in range(256) for g in range(256) for b in range(256)]

# محدود کردن تعداد رنگ‌ها به مقدار مشخص (max_colors)
selected_colors = all_colors[:max_colors]

# ذخیره هر ترکیب رنگ به‌عنوان یک تصویر
for i, color in enumerate(selected_colors):
    # ایجاد تصویر جدید
    img = Image.new("RGB", base_size, color)

    # تغییر اندازه تصویر
    img = img.resize(final_size, Image.NEAREST)

    # ذخیره تصویر با نام یکتا
    file_name = f"color_{i+1:05}.png"
    img.save(os.path.join(output_folder, file_name))

print(f"{len(selected_colors)} رنگ در پوشه '{output_folder}' ذخیره شد.")

200000 رنگ در پوشه 'data_color_2' ذخیره شد.


In [25]:
# پوشه حاوی تصاویر اصلی
input_folder = "data_color_2"
# پوشه برای ذخیره تصاویر سیاه و سفید
output_folder = "data_gray_2"
os.makedirs(output_folder, exist_ok=True)

# پردازش تمام تصاویر در پوشه ورودی
for file_name in os.listdir(input_folder):
    # مسیر کامل فایل ورودی
    input_path = os.path.join(input_folder, file_name)
    
    # بررسی اینکه فایل یک تصویر است
    if file_name.endswith(".png"):
        # باز کردن تصویر
        img = Image.open(input_path)
        
        # تبدیل به سیاه و سفید
        grayscale_img = img.convert("L")
        
        # مسیر کامل فایل خروجی
        output_path = os.path.join(output_folder, file_name)
        
        # ذخیره تصویر سیاه و سفید
        grayscale_img.save(output_path)

print(f"تمام تصاویر سیاه و سفید در پوشه '{output_folder}' ذخیره شدند.")

تمام تصاویر سیاه و سفید در پوشه 'data_gray_2' ذخیره شدند.


In [2]:
# 1. مسیر پوشه‌ها
input_folder = "data_gray_2"  # تصاویر سیاه‌وسفید
target_folder = "data_color_2"  # تصاویر رنگی

In [3]:
# 2. خواندن و پیش‌پردازش داده‌ها
def load_data(input_folder, target_folder, img_size=(2, 2)):
    X, Y = [], []
    for file_name in tqdm(os.listdir(input_folder)):
        # بررسی فایل‌های تصویری
        if file_name.endswith(".png"):
            # بارگذاری تصویر سیاه‌وسفید (ورودی)
            gray_img = load_img(os.path.join(input_folder, file_name), color_mode="grayscale", target_size=img_size)
            gray_img = img_to_array(gray_img) / 255.0  # نرمال‌سازی به [0, 1]
            X.append(gray_img)

            # بارگذاری تصویر رنگی (هدف)
            color_img = load_img(os.path.join(target_folder, file_name), target_size=img_size)
            color_img = img_to_array(color_img) / 255.0  # نرمال‌سازی به [0, 1]
            Y.append(color_img)
    
    return np.array(X), np.array(Y)

In [4]:
# بارگذاری داده‌ها
img_size = (32, 32)
X, Y = load_data(input_folder, target_folder, img_size)

100%|█████████████████████████████████████████████████████████████████████████| 200000/200000 [11:50<00:00, 281.41it/s]


In [5]:
# افزودن یک بعد به تصاویر سیاه‌وسفید (برای کانال‌ها)
X = X.reshape(-1, img_size[0], img_size[1], 1)

In [6]:
# تقسیم داده‌ها به آموزش و تست
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [7]:
# تابع ساخت مدل CNN برای رنگ‌آمیزی تصاویر
def create_cnn_colorization_model(input_shape=(32, 32, 1)):
    # ورودی مدل
    inputs = Input(shape=input_shape)

    # Encoder: استخراج ویژگی‌ها
    x = Conv2D(64, (3, 3), strides=1, padding="same")(inputs)
    x = BatchNormalization()(x)  # نرمال‌سازی دسته
    x = Activation("relu")(x)

    x = Conv2D(128, (3, 3), strides=2, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(256, (3, 3), strides=2, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    # Decoder: بازسازی تصویر رنگی
    x = Conv2D(128, (3, 3), strides=1, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = UpSampling2D()(x)

    x = Conv2D(64, (3, 3), strides=1, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = UpSampling2D()(x)

    # خروجی مدل که کانال‌های رنگی را پیش‌بینی می‌کند
    x = Conv2D(3, (3, 3), strides=1, padding="same")(x)  # 2 کانال برای رنگ (a, b) در فضای Lab
    output = Activation("tanh")(x)  # خروجی در بازه [-1, 1]

    # ساخت مدل
    model = Model(inputs, output)
    
    return model

In [8]:
model_1 =  create_cnn_colorization_model()
# نمایش معماری مدل
model_1.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 32, 32, 1)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 32, 32, 64)          │             640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 32, 32, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 32, 32, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 16, 16, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 16, 16, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 16, 16, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 8, 8, 256)           │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 8, 8, 256)           │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_2 (Activation)            │ (None, 8, 8, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 8, 8, 128)           │         295,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 8, 8, 128)           │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_3 (Activation)            │ (None, 8, 8, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ up_sampling2d (UpSampling2D)         │ (None, 16, 16, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 16, 16, 64)          │          73,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 16, 16, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_4 (Activation)            │ (None, 16, 16, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ up_sampling2d_1 (UpSampling2D)       │ (None, 32, 32, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 32, 32, 3)           │           1,7

 Total params: 742,787 (2.83 MB)

 Trainable params: 741,507 (2.83 MB)

 Non-trainable params: 1,280 (5.00 KB)

In [9]:
model_1.compile(optimizer="adam", loss="mse", metrics=["accuracy"])

In [10]:
# 4. آموزش مدل
model_1.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=4, batch_size=128)

Epoch 1/4
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 305s 240ms/step - accuracy: 0.6861 - loss: 0.0385 - val_accuracy: 0.7056 - val_loss: 0.0303
Epoch 2/4
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 298s 238ms/step - accuracy: 0.7015 - loss: 0.0277 - val_accuracy: 0.7068 - val_loss: 0.0265
Epoch 3/4
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 299s 239ms/step - accuracy: 0.7015 - loss: 0.0274 - val_accuracy: 0.7053 - val_loss: 0.0528
Epoch 4/4
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 295s 236ms/step - accuracy: 0.7031 - loss: 0.0271 - val_accuracy: 0.7023 - val_loss: 0.0271


In [11]:
# ارزیابی مدل با داده‌های تست
loss = model_1.evaluate(X_test, Y_test)
print(f'Test Loss: {loss}')

1250/1250 ━━━━━━━━━━━━━━━━━━━━ 16s 12ms/step - accuracy: 0.7037 - loss: 0.0271
Test Loss: [0.027142388746142387, 0.7023308873176575]


In [12]:
# ذخیره مدل
model_1.save('model_10.2.keras')

# بارگذاری مدل
model_30 = tf.keras.models.load_model('model_10.2.keras')

In [14]:
## بارگذاری تصویر رنگی
img = load_img('25.png', target_size=(32, 32))  # مسیر تصویر خود را وارد کنید
# تبدیل تصویر به سیاه و سفید
img_gray = img.convert('L')  # تبدیل به حالت Grayscale (سیاه و سفید)
# تبدیل تصویر به آرایه numpy
img_array = img_to_array(img_gray)

# نرمال‌سازی تصویر به بازه [0, 1]
img_array = img_array / 255.0

# تغییر ابعاد تصویر به (1, 128, 128, 1) برای ورود به مدل (اگر ورودی مدل شما تک کاناله است)
img_array = np.expand_dims(img_array, axis=-1)  # افزودن بعد کانال برای سیاه و سفید
# پیش‌بینی با مدل
prediction = model_30.predict(np.expand_dims(img_array, axis=0))

# بازگرداندن مقادیر پیش‌بینی شده به بازه [0, 255]
predicted_image = prediction[0] * 255.0
predicted_image = np.squeeze(predicted_image).astype(np.uint8)  # تبدیل به آرایه صحیح 8 بیتی

# ساخت تصویر از آرایه پیش‌بینی شده
predicted_img = Image.fromarray(predicted_image)

# ذخیره تصویر پیش‌بینی شده
output_folder = 'out put'  # مسیر پوشه‌ای که می‌خواهید تصویر ذخیره شود
os.makedirs(output_folder, exist_ok=True)  # اگر پوشه وجود ندارد، ایجاد می‌شود

output_image_path = os.path.join(output_folder, 'predicted_image_model_10.1.png')
predicted_img.save(output_image_path)

print(f"Predicted image saved at: {output_image_path}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Predicted image saved at: out put\predicted_image_model_10.1.png
